In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
import time
import matplotlib.pyplot as plt

# KMA API SETTINGS

API_KEY = "YnURiJMqTmm1EYiTKt5pNw"   # ← your authKey
LAT = 37.57141
LON = 126.96579
KST = pytz.timezone("Asia/Seoul")



# GET VALID KST TIME

def kst(minutes_ago=0):
    t = datetime.now(KST) - timedelta(minutes=minutes_ago)
    return t.strftime("%Y%m%d%H%M")



# PARSE FLOAT SAFELY

def parse_value(text):
    try:
        lines = text.strip().split("\n")
        for line in lines:
            try:
                return float(line.strip())
            except:
                pass
    except:
        pass
    return None



# FETCH 1 VARIABLE WITH DEBUG

def fetch_var(obs, tm):
    url = "https://apihub.kma.go.kr/api/typ01/url/sfc_nc_var.php"
    params = {
        "obs": obs,
        "lat": LAT,
        "lon": LON,
        "tm1": tm,
        "tm2": tm,
        "itv": 1,
        "authKey": API_KEY
    }

    print(f"\n📡 Fetching {obs} at {tm}")
    r = requests.get(url, params=params)

    print("Status:", r.status_code)
    print("RAW:", r.text)
    print("--------------------------------------")

    if r.status_code != 200:
        return None

    return parse_value(r.text)



# RISK EQUATIONS

def compute_risks(ta, hm, td, ws, ps, rain60, rainday):
    flood = 0.5 * hm + 0.4 * rain60 + 0.6 * rainday
    fire = 0.7 * (100 - hm) + 0.3 * ws
    moisture = 0.5 * hm + 0.3 * ta + 0.2 * td
    return flood, fire, moisture



# COLLECT DATA FOR N TIME POINTS

def collect_data(n_samples=10):
    rows = []

    for i in range(n_samples):
        tm = kst(i * 10)
        print("\n==============================")
        print("   Time:", tm)
        print("==============================")

        obs_list = {
            "ta": "Temperature",
            "hm": "Humidity",
            "td": "DewPoint",
            "ws_10m": "Wind10m",
            "ps": "Pressure",
            "rn_60m": "Rain60m",
            "rn_day": "RainDay"
        }

        vals = {}
        missing = False

        for obs, label in obs_list.items():
            v = fetch_var(obs, tm)
            vals[obs] = v
            print(f"{label}: {v}")
            if v is None:
                missing = True
            time.sleep(0.3)

        if missing:
            print("❌ Missing data → Skipped")
            continue

        flood, fire, moisture = compute_risks(
            vals["ta"], vals["hm"], vals["td"], vals["ws_10m"],
            vals["ps"], vals["rn_60m"], vals["rn_day"]
        )

        rows.append({
            "time": tm,
            "flood_risk": flood,
            "fire_risk": fire,
            "moisture_risk": moisture
        })

    return pd.DataFrame(rows)

# RUN

df = collect_data(10)

print("\nFINAL DATA:")
print(df)

# Save CSV
df.to_csv("risk_dataset.csv", index=False)
print("\n💾 Saved → risk_dataset.csv")


# PLOT RESULTS

if not df.empty:
    plt.figure(figsize=(12, 6))

    plt.plot(df["time"], df["flood_risk"], label="Flood Risk")
    plt.plot(df["time"], df["fire_risk"], label="Fire Risk")
    plt.plot(df["time"], df["moisture_risk"], label="Moisture Risk")

    plt.xlabel("Time")
    plt.ylabel("Risk Level")
    plt.title("Flood, Fire, and Moisture Risks Over Time")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("⚠ No data collected → cannot plot.")



   Time: 202511171033

📡 Fetching ta at 202511171033
Status: 403
RAW: {
  "result" : {
    "status" : 403,
    "message" : "활용신청이 필요한 API 입니다. 활용신청 후 다시 시도해 주십시오."
  }
}
--------------------------------------
Temperature: None

📡 Fetching hm at 202511171033
Status: 403
RAW: {
  "result" : {
    "status" : 403,
    "message" : "활용신청이 필요한 API 입니다. 활용신청 후 다시 시도해 주십시오."
  }
}
--------------------------------------
Humidity: None

📡 Fetching td at 202511171033
Status: 403
RAW: {
  "result" : {
    "status" : 403,
    "message" : "활용신청이 필요한 API 입니다. 활용신청 후 다시 시도해 주십시오."
  }
}
--------------------------------------
DewPoint: None

📡 Fetching ws_10m at 202511171033
Status: 403
RAW: {
  "result" : {
    "status" : 403,
    "message" : "활용신청이 필요한 API 입니다. 활용신청 후 다시 시도해 주십시오."
  }
}
--------------------------------------
Wind10m: None

📡 Fetching ps at 202511171033
Status: 403
RAW: {
  "result" : {
    "status" : 403,
    "message" : "활용신청이 필요한 API 입니다. 활용신청 후 다시 시도해 주십시오."
  }
}
---------------